In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.2 MB/s eta 0:00:00


# 1. Reading in the data

In [2]:
import json
root_path = "/content/drive/MyDrive/emotifymusic"
with open(root_path+"/labels.json", 'r') as f:
    labels = json.load(f)

In [3]:
#mapping = [' amazement', ' solemnity', ' tenderness',' nostalgia', ' calmness', ' power', ' joyful', ' tension',' sadness']
import pandas as pd
labels = pd.DataFrame(labels)
labels.head()

,file_path,output_path,label
0,classical/1.mp3,classical_1.mp3,4
1,classical/1.mp3,classical_1.mp3,4
2,classical/1.mp3,classical_1.mp3,4
3,classical/1.mp3,classical_1.mp3,4
4,classical/1.mp3,classical_1.mp3,4


In [5]:
import numpy as np
path = "/content/drive/MyDrive/emotifymusic/spectrograms/"
log_spectrograms = []
for index,rows in labels.iterrows():
  spec_path =  path+rows["output_path"]+".npy"
  spec = np.load(spec_path)
  log_spectrograms.append(spec)
len(log_spectrograms)


2385

In [7]:
labels["log_spectrogram"] = log_spectrograms
labels.head()

,file_path,output_path,label,log_spectrogram
0,classical/1.mp3,classical_1.mp3,4,"[[-23.430485, -35.5547, -31.009932, -33.170452..."
1,classical/1.mp3,classical_1.mp3,4,"[[-23.430485, -35.5547, -31.009932, -33.170452..."
2,classical/1.mp3,classical_1.mp3,4,"[[-23.430485, -35.5547, -31.009932, -33.170452..."
3,classical/1.mp3,classical_1.mp3,4,"[[-23.430485, -35.5547, -31.009932, -33.170452..."
4,classical/1.mp3,classical_1.mp3,4,"[[-23.430485, -35.5547, -31.009932, -33.170452..."


In [98]:
# input shape 1024*431
list(labels["log_spectrogram"])[0][1][1]

-23.384499

# 2. Train test split

In [102]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

X = np.array(list(labels["log_spectrogram"])).astype("float32")
X = X[...,np.newaxis]
y = np.array(labels["label"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



431

In [104]:
X_train.shape

(1908, 1024, 431, 1)

# 3. Builiding the basic A2E model


In [105]:
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, (5, 5), strides=2, activation="relu", padding="valid",input_shape = (X_train.shape[1],X_train.shape[2],1)))
model.add(keras.layers.BatchNormalization())

# 2nd Layer
model.add(keras.layers.Conv2D(64, (3, 3), strides=1, activation="relu", padding="same"))
model.add(keras.layers.BatchNormalization())

# 3rd Layer
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.3))

# 4th Layer
model.add(keras.layers.Conv2D(128, (3, 3), strides=1, activation="relu", padding="same"))
model.add(keras.layers.BatchNormalization())

# 5th Layer
model.add(keras.layers.Conv2D(128, (3, 3), strides=1, activation="relu", padding="same"))
model.add(keras.layers.BatchNormalization())

# 6th Layer
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.3))

# 7th Layer
model.add(keras.layers.Conv2D(256, (3, 3), strides=1, activation="relu", padding="same"))
model.add(keras.layers.BatchNormalization())

# 8th Layer
model.add(keras.layers.Conv2D(256, (3, 3), strides=1, activation="relu", padding="same"))
model.add(keras.layers.BatchNormalization())

# 9th Layer
model.add(keras.layers.Conv2D(384, (3, 3), strides=1, activation="relu", padding="same"))
model.add(keras.layers.BatchNormalization())

# 10th Layer
model.add(keras.layers.Conv2D(512, (3, 3), strides=1, activation="relu", padding="same"))
model.add(keras.layers.BatchNormalization())

# 11th Layer
model.add(keras.layers.Conv2D(256, (3, 3), strides=1, activation="relu", padding="same"))
model.add(keras.layers.BatchNormalization())

# 12th Layer
model.add(tfa.layers.AdaptiveAveragePooling2D((1,1)))

# 13th layer Dense
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(9,activation="softmax"))

# 4. Training the model

In [106]:
optimiser = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimiser,loss="sparse_categorical_crossentropy",metrics=["accuracy"])
history_model = model.fit(X_train, y_train, epochs = 50, batch_size=8, validation_data=(X_test,y_test))
model.summary()

Epoch 1/50
239/239 [==============================] - 90s 299ms/step - loss: 2.0142 - accuracy: 0.3061 - val_loss: 2.3280 - val_accuracy: 0.2579
Epoch 2/50
239/239 [==============================] - 63s 265ms/step - loss: 1.8158 - accuracy: 0.3585 - val_loss: 2.1516 - val_accuracy: 0.2725
Epoch 3/50
239/239 [==============================] - 63s 264ms/step - loss: 1.7612 - accuracy: 0.3606 - val_loss: 2.1598 - val_accuracy: 0.2075
Epoch 4/50
239/239 [==============================] - 63s 264ms/step - loss: 1.7018 - accuracy: 0.3973 - val_loss: 1.7423 - val_accuracy: 0.3857
Epoch 5/50
239/239 [==============================] - 63s 264ms/step - loss: 1.6360 - accuracy: 0.4256 - val_loss: 2.2374 - val_accuracy: 0.2851
Epoch 6/50
239/239 [==============================] - 63s 264ms/step - loss: 1.5845 - accuracy: 0.4513 - val_loss: 1.5870 - val_accuracy: 0.4444
Epoch 7/50
239/239 [==============================] - 63s 264ms/step - loss: 1.5110 - accuracy: 0.4869 - val_loss: 1.7695 - val_ac

In [108]:
model.save('/content/drive/MyDrive/emotifymusic/my_model.keras')